In [1]:
!pip install filterpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110458 sha256=e735e9d90f8422680dbafc9ab3586b0a3ccae20a47861e73135529c226f0363b
  Stored in directory: /root/.cache/pip/wheels/12/dc/3c/e12983eac132d00f82a20c6cbe7b42ce6e96190ef8fa2d15e1
Successfully built filterpy


In [2]:
!pip install chess


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 45.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147776 sha256=81f6bdeebfc8713cd00d1912c36f5b21792df1c32bbea12066fa789e2996064a
  Stored in directory: /root/.cache/pip/wheels/fb/5d/5c/59a62d8a695285e59ec9c1f66add6f8a9ac4152499a2be0113
Successfully built chess


In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [4]:
import cv2
import numpy as np
import chess
from ultralytics import YOLO
import logging
from collections import deque

class ChessMoveDetector:
    def __init__(self, model_path, video_path, output_video_path="detected_moves.mp4"):
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

        try:
            self.model = YOLO(model_path)
            self.logger.info(f"Model loaded successfully: {model_path}")
        except Exception as e:
            self.logger.error(f"Failed to load model: {e}")
            raise

        self.video_path = video_path
        self.output_video_path = output_video_path
        self.board_history = deque(maxlen=5)  # Store last 5 board positions
        self.piece_persistence = {}  # Store detected pieces for tracking

        self.piece_map = {
            'black-pawn': (chess.PAWN, chess.BLACK),
            'white-pawn': (chess.PAWN, chess.WHITE),
            'black-rook': (chess.ROOK, chess.BLACK),
            'white-rook': (chess.ROOK, chess.WHITE),
            'black-knight': (chess.KNIGHT, chess.BLACK),
            'white-knight': (chess.KNIGHT, chess.WHITE),
            'black-bishop': (chess.BISHOP, chess.BLACK),
            'white-bishop': (chess.BISHOP, chess.WHITE),
            'black-queen': (chess.QUEEN, chess.BLACK),
            'white-queen': (chess.QUEEN, chess.WHITE),
            'black-king': (chess.KING, chess.BLACK),
            'white-king': (chess.KING, chess.WHITE)
        }

    def extract_frames(self, interval=1):
        vidcap = cv2.VideoCapture(self.video_path)
        if not vidcap.isOpened():
            raise ValueError(f"Error: Could not open video at {self.video_path}")

        frames, timestamps = [], []
        fps = vidcap.get(cv2.CAP_PROP_FPS)
        success, image = vidcap.read()
        count = 0

        while success:
            if count % int(fps * interval) == 0:
                frames.append(image)
                timestamps.append(count / fps)
            success, image = vidcap.read()
            count += 1

        vidcap.release()
        self.logger.info(f"Extracted {len(frames)} frames")
        return frames, timestamps, fps

    def detect_pieces(self, frame):
        try:
            results = self.model.predict(source=frame, imgsz=640, conf=0.35)
            detected_pieces = []

            for r in results[0].boxes:
                class_id = int(r.cls[0])
                class_name = self.model.names[class_id]
                x, y, w, h = r.xywh[0]

                detected_pieces.append({'name': class_name, 'x': x.item(), 'y': y.item()})

            return detected_pieces
        except Exception as e:
            self.logger.error(f"Piece detection error: {e}")
            return []

    def frame_to_board(self, frame):
        board = chess.Board()
        board.clear()

        detected_pieces = self.detect_pieces(frame)
        frame_height, frame_width = frame.shape[:2]

        for piece in detected_pieces:
            try:
                piece_type, color = self.piece_map.get(piece['name'], (None, None))
                if piece_type is None:
                    continue

                square = self.map_to_board_square(piece['x'], piece['y'], frame_width, frame_height)
                board.set_piece_at(square, chess.Piece(piece_type, color))
            except Exception as e:
                self.logger.error(f"Error processing piece {piece}: {e}")

        self.board_history.append(board)
        return board, detected_pieces

    def map_to_board_square(self, x, y, frame_width, frame_height):
        col = int(x / (frame_width / 8))
        row = 7 - int(y / (frame_height / 8))
        return chess.square(col, row)

    def validate_move(self, board, move_uci):
        """
        Check if the detected move is valid.
        """
        return move_uci in [m.uci() for m in board.legal_moves]

    def get_moves(self, board_positions):
        """
        Compare board positions to detect valid moves.
        """
        moves = []
        seen_moves = set()  # Prevent duplicates

        for i in range(1, len(board_positions)):
            prev_board = board_positions[i - 1]
            curr_board = board_positions[i]

            move_found = False
            best_match = None

            for move in prev_board.legal_moves:
                test_board = prev_board.copy()
                test_board.push(move)

                if sum(1 for sq in chess.SQUARES if test_board.piece_at(sq) != curr_board.piece_at(sq)) <= 4:
                    if move.uci() not in seen_moves and self.validate_move(prev_board, move.uci()):
                        moves.append(move.uci())
                        seen_moves.add(move.uci())
                        move_found = True
                        break
                    elif best_match is None:
                        best_match = move.uci()

            if not move_found and best_match:
                moves.append(best_match)

        return moves

    def draw_detections(self, frame, detected_pieces):
        """
        Draw bounding boxes and detected pieces on the frame.
        """
        for piece in detected_pieces:
            x, y = int(piece['x']), int(piece['y'])
            cv2.circle(frame, (x, y), 10, (0, 255, 0), -1)
            cv2.putText(frame, piece['name'], (x - 15, y - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        return frame

    def process_video(self, interval=2):
        try:
            frames, timestamps, fps = self.extract_frames(interval)
            board_positions = []
            processed_frames = []

            for frame in frames:
                board, detected_pieces = self.frame_to_board(frame)
                board_positions.append(board)

                processed_frame = self.draw_detections(frame, detected_pieces)
                processed_frames.append(processed_frame)

            moves = self.get_moves(board_positions)
            self.logger.info(f"Detected {len(moves)} moves")

            self.save_video(processed_frames, fps)
            return moves
        except Exception as e:
            self.logger.error(f"Video processing error: {e}")
            return []

    def save_video(self, frames, fps):
        """
        Save processed frames as a video.
        """
        height, width, layers = frames[0].shape
        video_writer = cv2.VideoWriter(self.output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

        for frame in frames:
            video_writer.write(frame)

        video_writer.release()
        self.logger.info(f"Video saved successfully as {self.output_video_path}")

def main():
    MODEL_PATH = "/content/drive/MyDrive/Chess_Model/best.pt"
    VIDEO_PATH = "/content/drive/MyDrive/Chess_Model/How To Play The Queen's Gambit.mp4"
    OUTPUT_VIDEO_PATH = "/content/drive/MyDrive/Chess_Model/detected_moves.mp4"

    try:
        detector = ChessMoveDetector(MODEL_PATH, VIDEO_PATH, OUTPUT_VIDEO_PATH)
        moves = detector.process_video()

        with open("/content/drive/MyDrive/Chess_Model/moves.txt", "w") as f:
            f.write("\n".join(moves))

        print(f"Moves detected and saved successfully! Video saved as {OUTPUT_VIDEO_PATH}")

    except Exception as e:
        print(f"Error in main processing: {e}")

if __name__ == "__main__":
    main()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

0: 384x640 3 black-knights, 8 black-pawns, 1 black-rook, 6 white-knights, 6 white-pawns, 44.1ms
Speed: 7.8ms preprocess, 44.1ms inference, 362.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 black-knights, 8 black-pawns, 1 black-rook, 6 white-knights, 6 white-pawns, 10.8ms
Speed: 2.4ms preprocess, 10.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 black-knight, 7 black-pawns, 6 white-knights, 7 white-pawns, 10.7ms
Speed: 1.9ms preprocess, 10.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 black-knight, 7 black-pawns, 6 white-knights, 7 white-pawns, 15.2ms
Speed: 1.6ms preprocess, 15.2ms infere

In [5]:
import os
os.environ["OPENAI_API_KEY"] = "api_key"


In [6]:
import openai
import os

# Set OpenAI API key securely
openai.api_key = os.getenv("OPENAI_API_KEY")  # Ensure this is set in your environment

if not openai.api_key:
    raise ValueError("❌ API Key Missing! Set it in your environment variables.")

# Load the chess analysis moves
file_path = "/content/drive/MyDrive/Chess_Model/moves1.txt"

if not os.path.exists(file_path):
    raise FileNotFoundError("❌ File not found! Make sure 'chess_analysis_llm.txt' exists.")

with open(file_path, "r") as f:
    moves_text = f.read()

# Define LLM Prompt
llm_prompt = f"""
You are a witty chess commentator, blending deep strategic insight with humor.
Analyze the following chess moves and provide a funny yet insightful commentary:

Moves:
{moves_text}

Your response should include:
1. A short summary of the game so far.
2. Analysis of the players' strategies.
3. Humorous remarks on questionable or bold moves.

Keep it entertaining but informative!
"""

# ✅ Use the New OpenAI API Format
client = openai.OpenAI()

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a witty chess commentator providing humorous and insightful game analysis."},
        {"role": "user", "content": llm_prompt}
    ]
)

# Extract & Save Commentary
llm_commentary = response.choices[0].message.content

with open("/content/chess_analysis_llm.txt", "w") as f:
    f.write(llm_commentary)

print("✅ Chess commentary generated and saved to 'chess_analysis_llm.txt'!")


✅ Chess commentary generated and saved to 'chess_analysis_llm.txt'!


In [7]:
!pip uninstall numpy pandas -y
!pip install --upgrade numpy pandas
!pip install --upgrade tts


Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 110.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ultralytics 8.3.96 requires numpy<=2.1.1,>=1.23.0, but you have numpy 2.2.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
num

In [8]:
!pip install pydub

In [9]:
!pip install --upgrade tts soundfile pydub


In [10]:
!pip install gtts


In [11]:
from gtts import gTTS

# Path to the generated LLM commentary file
llm_commentary_path = "/content/chess_analysis_llm.txt"  # Change this path if needed

# Read the commentary text
with open(llm_commentary_path, "r") as f:
    commentary_text = f.read()

# Convert the text to speech using gTTS
tts = gTTS(text=commentary_text, lang="en", slow=False)

# Save the audio file
audio_output_path = "chess_analysis.mp3"  # Change this path if needed
tts.save(audio_output_path)

print(f"✅ Chess commentary saved as '{audio_output_path}'")


✅ Chess commentary saved as 'chess_analysis.mp3'


In [12]:
!pip install streamlit openai gtts chess pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.7 MB/s eta 0:00:00


In [13]:
%%writefile app.py
# Paste the Streamlit code here (from above)
import streamlit as st
import openai
import os
import chess
import tempfile
from gtts import gTTS
import shutil

# Set OpenAI API key (Replace with your actual key or use environment variable)
openai.api_key = os.getenv("OPENAI_API_KEY", "openai_key")

st.title("♟️ Chess Video Analyzer with AI Commentary 🎙️")

# Upload video file
uploaded_file = st.file_uploader("📂 Upload a Chess Video", type=["mp4", "mov", "avi"])

if uploaded_file:
    temp_dir = tempfile.mkdtemp()
    video_path = os.path.join(temp_dir, uploaded_file.name)

    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.success("✅ Video uploaded successfully!")
    st.video(video_path)
    st.subheader("🎯 Detected Chess Moves Video:")
    st.video("/content/drive/MyDrive/Chess_Model/detected_moves.mp4")


    # Simulated Chess Move Detection (Replace with real model later)
    def detect_moves():
      with open("/content/drive/MyDrive/Chess_Model/moves1.txt", "r") as f:
        moves = [line.strip() for line in f if line.strip()]
        return moves


    detected_moves = detect_moves()

    st.subheader("♟️ Detected Moves:")
    st.write(", ".join(detected_moves))

    # Generate LLM Commentary
    st.info("🤖 Generating Chess Commentary...")

    prompt = f"""
    You are a witty chess commentator providing humorous insights.
    Given these chess moves, generate a funny and insightful game analysis:

    Moves:
    {', '.join(detected_moves)}

    Response should include:
    1. A short summary of the game so far.
    2. Analysis of the players' strategies.
    3. Humorous remarks.

    Keep it entertaining but informative!
    """

    client = openai.OpenAI()
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a witty chess commentator."},
            {"role": "user", "content": prompt}
        ]
    )

    chess_commentary = response.choices[0].message.content

    st.subheader("🎭 Humorous Chess Commentary:")
    st.write(chess_commentary)

    # Convert Commentary to Speech (Using gTTS)
    st.info("🎙️ Converting commentary to speech...")

    try:
        audio_path = os.path.join(temp_dir, "chess_commentary.mp3")
        tts = gTTS(text=chess_commentary, lang="en", slow=False)
        tts.save(audio_path)

        # Display Download Button
        st.audio(audio_path)

    except Exception as e:
        st.error(f"⚠️ gTTS failed: {e}")

    # Cleanup temporary files
    shutil.rmtree(temp_dir)

    st.success("✅ Chess analysis completed!")


Writing app.py


In [18]:
!curl https://loca.lt/mytunnelpassword

35.227.158.134

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.227.158.134:8501

⠼⠴Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙your url is: https://two-mirrors-notice.loca.lt
